<a href="https://colab.research.google.com/github/nandyceg/Bioactivity-Prediction/blob/main/Lipinski_descriptor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2021-10-28 11:36:05--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M   130MB/s    in 0.6s    

2021-10-28 11:36:06 (130 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h

In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

In [ ]:
df = pd.read_csv('bioactivity_preprocessed.csv')

In [ ]:
m=Chem.MolFromSmiles(df['canonical_smiles'][5])
m

In [ ]:
def lipinski(smiles, verbose=False):

    mol_data= []    
    for element in smiles:
     
      try:
          mol=Chem.MolFromSmiles(element) 
          mol_data.append(mol)
      except:
          print('Invalid SMILES:', element)
        
       
    base_data= np.arange(1,1)
    i=0  
    for mol in mol_data:        
       
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
           
        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])   
    
        if(i==0):
            base_data=row
        else:
            base_data=np.vstack([base_data, row])
        i=i+1      
    
    column_names=["MW","LogP","HBD","HBA"]   
    descriptors = pd.DataFrame(data=base_data,columns=column_names)
    
    return descriptors

In [ ]:
df_lp = lipinski(df.canonical_smiles)
df_lp

,MW,LogP,HBD,HBA
0,312.325,2.80320,0.0,6.0
1,376.913,4.55460,0.0,5.0
2,426.851,5.35740,0.0,5.0
3,404.845,4.70690,0.0,5.0
4,346.334,3.09530,0.0,6.0
...,...,...,...,...
5198,453.583,3.79622,2.0,7.0
5199,474.001,4.14120,2.0,7.0
5200,518.452,4.25030,2.0,7.0
5201,457.546,3.62690,2.0,7.0


In [ ]:
df_withlp = pd.concat([df,df_lp], axis=1)

In [ ]:
df

,molecule_chembl_id,canonical_smiles,standard_value,bioactivity_status
0,CHEMBL133897,CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1,750.0,active
1,CHEMBL336398,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1,100.0,active
2,CHEMBL131588,CN(C(=O)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F)c1ccccc1,50000.0,inactive
3,CHEMBL130628,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F,300.0,active
4,CHEMBL130478,CSc1nc(-c2ccc(OC(F)(F)F)cc2)nn1C(=O)N(C)C,800.0,active
...,...,...,...,...
5198,CHEMBL4089550,COc1cc2c(cc1OC)CN(CC1CCN(Cc3cc(C)cc(/C=N/O)c3O...,417500.0,active
5199,CHEMBL4102122,COc1cc2c(cc1OC)CN(CC1CCN(Cc3cc(Cl)cc(/C=N/O)c3...,636700.0,active
5200,CHEMBL4077866,COc1cc2c(cc1OC)CN(CC1CCN(Cc3cc(Br)cc(/C=N/O)c3...,568000.0,inactive
5201,CHEMBL4064154,COc1cc2c(cc1OC)CN(CC1CCN(Cc3cc(F)cc(/C=N/O)c3O...,658100.0,inactive


In [ ]:
df_lp

,MW,LogP,HBD,HBA
0,312.325,2.80320,0.0,6.0
1,376.913,4.55460,0.0,5.0
2,426.851,5.35740,0.0,5.0
3,404.845,4.70690,0.0,5.0
4,346.334,3.09530,0.0,6.0
...,...,...,...,...
5198,453.583,3.79622,2.0,7.0
5199,474.001,4.14120,2.0,7.0
5200,518.452,4.25030,2.0,7.0
5201,457.546,3.62690,2.0,7.0


In [ ]:
df_withlp

,molecule_chembl_id,canonical_smiles,standard_value,bioactivity_status,MW,LogP,HBD,HBA
0,CHEMBL133897,CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1,750.0,active,312.325,2.80320,0.0,6.0
1,CHEMBL336398,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1,100.0,active,376.913,4.55460,0.0,5.0
2,CHEMBL131588,CN(C(=O)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F)c1ccccc1,50000.0,inactive,426.851,5.35740,0.0,5.0
3,CHEMBL130628,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F,300.0,active,404.845,4.70690,0.0,5.0
4,CHEMBL130478,CSc1nc(-c2ccc(OC(F)(F)F)cc2)nn1C(=O)N(C)C,800.0,active,346.334,3.09530,0.0,6.0
...,...,...,...,...,...,...,...,...
5198,CHEMBL4089550,COc1cc2c(cc1OC)CN(CC1CCN(Cc3cc(C)cc(/C=N/O)c3O...,417500.0,active,453.583,3.79622,2.0,7.0
5199,CHEMBL4102122,COc1cc2c(cc1OC)CN(CC1CCN(Cc3cc(Cl)cc(/C=N/O)c3...,636700.0,active,474.001,4.14120,2.0,7.0
5200,CHEMBL4077866,COc1cc2c(cc1OC)CN(CC1CCN(Cc3cc(Br)cc(/C=N/O)c3...,568000.0,inactive,518.452,4.25030,2.0,7.0
5201,CHEMBL4064154,COc1cc2c(cc1OC)CN(CC1CCN(Cc3cc(F)cc(/C=N/O)c3O...,658100.0,inactive,457.546,3.62690,2.0,7.0


In [ ]:
df_withlp.standard_value.describe()

count    5.203000e+03
mean     1.650392e+13
std      9.879054e+14
min      6.000000e-03
25%      7.700000e+01
50%      1.000000e+03
75%      1.208907e+04
max      7.079458e+16
Name: standard_value, dtype: float64

In [ ]:
def pIC50(df_in):
  pIC50 =[]
  for i in df_in['standard_value_norm']:
    molar = i*(10**-9)
    pIC50.append(-np.log10(molar))
  df_in['pIC50'] = pIC50
  res = df_in.drop('standard_value_norm',1)
  return res

In [ ]:
def norm_val(df_input):
  norm=[]
  for i in df_input['standard_value']:
    if i>100000000:
      i=100000000
    norm.append(i)
  df_input['standard_value_norm'] = norm
  res = df_input.drop('standard_value',1)
  return res

In [ ]:
df_norm = norm_val(df_withlp)
df_norm

,molecule_chembl_id,canonical_smiles,bioactivity_status,MW,LogP,HBD,HBA,standard_value_norm
0,CHEMBL133897,CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1,active,312.325,2.80320,0.0,6.0,750.0
1,CHEMBL336398,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1,active,376.913,4.55460,0.0,5.0,100.0
2,CHEMBL131588,CN(C(=O)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F)c1ccccc1,inactive,426.851,5.35740,0.0,5.0,50000.0
3,CHEMBL130628,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F,active,404.845,4.70690,0.0,5.0,300.0
4,CHEMBL130478,CSc1nc(-c2ccc(OC(F)(F)F)cc2)nn1C(=O)N(C)C,active,346.334,3.09530,0.0,6.0,800.0
...,...,...,...,...,...,...,...,...
5198,CHEMBL4089550,COc1cc2c(cc1OC)CN(CC1CCN(Cc3cc(C)cc(/C=N/O)c3O...,active,453.583,3.79622,2.0,7.0,417500.0
5199,CHEMBL4102122,COc1cc2c(cc1OC)CN(CC1CCN(Cc3cc(Cl)cc(/C=N/O)c3...,active,474.001,4.14120,2.0,7.0,636700.0
5200,CHEMBL4077866,COc1cc2c(cc1OC)CN(CC1CCN(Cc3cc(Br)cc(/C=N/O)c3...,inactive,518.452,4.25030,2.0,7.0,568000.0
5201,CHEMBL4064154,COc1cc2c(cc1OC)CN(CC1CCN(Cc3cc(F)cc(/C=N/O)c3O...,inactive,457.546,3.62690,2.0,7.0,658100.0


In [ ]:
df_2 = pIC50(df_norm)
df_2

,molecule_chembl_id,canonical_smiles,bioactivity_status,MW,LogP,HBD,HBA,pIC50
0,CHEMBL133897,CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1,active,312.325,2.80320,0.0,6.0,6.124939
1,CHEMBL336398,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1,active,376.913,4.55460,0.0,5.0,7.000000
2,CHEMBL131588,CN(C(=O)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F)c1ccccc1,inactive,426.851,5.35740,0.0,5.0,4.301030
3,CHEMBL130628,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F,active,404.845,4.70690,0.0,5.0,6.522879
4,CHEMBL130478,CSc1nc(-c2ccc(OC(F)(F)F)cc2)nn1C(=O)N(C)C,active,346.334,3.09530,0.0,6.0,6.096910
...,...,...,...,...,...,...,...,...
5198,CHEMBL4089550,COc1cc2c(cc1OC)CN(CC1CCN(Cc3cc(C)cc(/C=N/O)c3O...,active,453.583,3.79622,2.0,7.0,3.379344
5199,CHEMBL4102122,COc1cc2c(cc1OC)CN(CC1CCN(Cc3cc(Cl)cc(/C=N/O)c3...,active,474.001,4.14120,2.0,7.0,3.196065
5200,CHEMBL4077866,COc1cc2c(cc1OC)CN(CC1CCN(Cc3cc(Br)cc(/C=N/O)c3...,inactive,518.452,4.25030,2.0,7.0,3.245652
5201,CHEMBL4064154,COc1cc2c(cc1OC)CN(CC1CCN(Cc3cc(F)cc(/C=N/O)c3O...,inactive,457.546,3.62690,2.0,7.0,3.181708


In [ ]:
df_2.to_csv('bioactivity_lipinski_pIC50.csv',index=0)

In [ ]:
ls -l

total 84148
-rw-r--r-- 1 root root   688460 Oct 28 11:41 bioactivity_lipinski_pIC50.csv
-rw-r--r-- 1 root root   412614 Oct 28 11:38 bioactivity_preprocessed.csv
-rwxr-xr-x 1 root root 85055499 Mar 11  2020 Miniconda3-py37_4.8.2-Linux-x86_64.sh*
drwxr-xr-x 1 root root     4096 Oct 26 13:34 sample_data/
